In [1]:
from pyspark import SparkContext

In [2]:
sc = SparkContext('local')

In [3]:
rdd1 = sc.textFile('../data1/bb.txt')
rdd1.collect()

['10,20', '30,40', '50,60', '20,40']

In [4]:
s = '10, 20'
s.split(',')

['10', ' 20']

In [6]:
rdd2 = rdd1.map(lambda v: v.split(','))
rdd2.collect() # 새로운 RDD가 반환

[['10', '20'], ['30', '40'], ['50', '60'], ['20', '40']]

In [7]:
rdd3 = rdd1.flatMap(lambda v: v.split(','))
rdd3.collect() # 새로운 RDD가 반환 / 현업에서는 collect함부로 사용하면 안 되고 take를 씀 -> 일부를 확인

['10', '20', '30', '40', '50', '60', '20', '40']

In [9]:
rdd4 = rdd3.map(lambda v: int(v)) # 문자열을 정수로 변환
rdd4.collect()

[10, 20, 30, 40, 50, 60, 20, 40]

In [10]:
# 1차 transfomation + 2차 transfomation
# action으로 끝났기 때문에 더 이상 이어 붙일 수 없음

rdd1.flatMap(lambda v: v.split(',')).map(lambda v: int(v)).collect()

[10, 20, 30, 40, 50, 60, 20, 40]

In [11]:
rdd5 = rdd4.distinct() # transformation함수 : 중복된 값 제거
rdd5.collect()

[10, 20, 30, 40, 50, 60]

In [13]:
rdd5.first() # action함수 : 첫번째 요소만 들고 옴

# action함수의 반환값은 정수나 리스트
# transformation함수의 반환값은 transformation

10

In [15]:
samRdd1 = rdd5.sample(False, 0.3) # 30프로를 비복원 추출
samRdd1.collect()

[10, 20, 50, 60]

In [16]:
samRdd2 = rdd5.sample(True, 0.3) # 복원 추출 - 중복된 값이 나올 수 있음
samRdd2.collect()

[40, 50]

In [19]:
samListrdd3 = rdd5.takeSample(False, 3) # action : sample 달리 그냥 sapsample은 퍼센트로 추출
# 비복원
samListrdd3 # action이기 때문에 바로 찍어야 함

[30, 40, 20]

In [20]:
samListrdd4 = rdd5.takeSample(True, 3) # action : # 복원
samListrdd4

[20, 30, 40]

In [21]:
# 통계관련 함수는 모두 action함수
rdd5.mean() # action 

35.0

In [22]:
rdd5.sum()

210

In [23]:
rdd5.max()

60

In [24]:
rdd5.stdev()

17.07825127659933

In [26]:
rdd5.histogram([1, 50, 100]) # 범위랑 카운트를 반환 # 1<=data<50 => 4개, 50<=data<100 => 2개

# 튜플에서 unpacking해서 도수를 구하면 됨

([1, 50, 100], [4, 2])

In [29]:
rdd5.collect()

[10, 20, 30, 40, 50, 60]

In [27]:
a, b = rdd5.histogram([1, 50, 100])
b

[4, 2]

In [28]:
rdd5.stats() # 가장 기본적인 기술 통게값이 출력

(count: 6, mean: 35.0, stdev: 17.07825127659933, max: 60.0, min: 10.0)

In [30]:
# 콘솔에 print한 내용이 찍힘
# 함수의 아규먼트는 두 개가 있어야 함

# 아래와 같이 더하기를 해 버리면 누적합

def fn(a,b):
    print( 'a=',a)
    print( 'b=',b)
    return a+b

In [31]:
rdd5.reduce(fn) # action함수 : 누적해서 뭔가를 할 때 reduce를 사용

210

In [32]:
# 혹은 람다 사용
# map은 transformation함수

rdd5.reduce(lambda a, b:a+b)

210